# Chap02 - More PyMC

## 2.1 서론

- `PyMC` 구문과 디자인 패턴에 대해 알 수 있다.
- 베이지안 관점에서 시스템을 모델링 하는 방법을 알 수 있다.
- 베이지안 모델의 적합도 평가 및 데이터 시각화 기법을 알 수 있다.

### 2.1.1 부모와 자식 관계

> **부모변수**(Parent variable)는 다른 변수에 영항을 주는 변수다.

> **자식변수**(Child variable)는 다른 변수의 영향을 받는 즉, 부모변수에 종속된다.

어느 변수라도 부모변수가 될 수 있고, 동시에 자식변수가 될 수 있다.

In [1]:
import pymc as pm
import matplotlib

matplotlib.rc('font', family='NanumGothic')  # linux
# matplotlib.rc('font', family='AppleGothic')  # Mac

lambda_ = pm.Exponential('poisson_param', 1)
# 다음 변수 호출에 사용된다.
# lambda_는 data_generator의 부모변수다.
data_generator = pm.Poisson('data_generator', lambda_)
# data_generator는 data_plus_one의 부모변수다
data_plus_one = data_generator + 1

In [2]:
print("Children of 'lambda_': \n", lambda_.children)
print("\nParents of 'data_generator': \n", data_generator.parents)
print("\nChildren of 'data_generator': \n", data_generator.children)

Children of 'lambda_': 
 {<pymc.distributions.new_dist_class.<locals>.new_class 'data_generator' at 0x7febd42f7da0>}

Parents of 'data_generator': 
 {'mu': <pymc.distributions.new_dist_class.<locals>.new_class 'poisson_param' at 0x7febd42f7e80>}

Children of 'data_generator': 
 {<pymc.PyMCObjects.Deterministic '(data_generator_add_1)' at 0x7febd42f7748>}


### 2.1.2 `PyMC` 변수

모든 `PyMC`변수는 `value` 속성을 가진다. 이 속성을 통해 변수의 **현재값**(random한 값)을 생성한다.

자식변수(child variable)의 값은 부모변수 값이 주어질 때 변경된다.

In [3]:
print('lambda_.value =', lambda_.value)
print('data_generator.value =', data_generator.value)
print('data_plus_one.value =', data_plus_one.value)

lambda_.value = 0.04330518392458977
data_generator.value = 0
data_plus_one.value = 1


`PyMC`은 `stochastic`과 `deterministic` 두 가지 변수를 가진다.
- `stochastic` 변수는 값이 정해지지 않은 변수로 Random(난수)이며 `Poisson, DiscreteUniform, Exponential` 클래스의 인스턴스가 있다.
- `deterministic` 변수는 고정된(?) 값이다. 

#### stochastic 변수 초기화

```python
some_variable = pm.DiscreteUniform('discrete_uni_var', 0, 4)
```

`0, 4`는 `DiscreteUniform`의 난수 하한선과 상한선이다. `name` arg는 분석 후반에 사후확률분포를 가져올 때 사용하므로 이름을 사용하는 것이 좋다.

In [4]:
# ??를 이용해 확인하기
??pm.DiscreteUniform

Init signature: pm.DiscreteUniform(*args, **kwds)
Docstring:     
D = DiscreteUniform(name, lower, upper, value=None, observed=False, size=1, trace=True, rseed=True, doc=None, verbose=-1, debug=False)

Stochastic variable with DiscreteUniform distribution.
Parents are: lower, upper.

Docstring of log-probability function:

    Discrete uniform log-likelihood.

    .. math::
        f(x \mid lower, upper) = \frac{1}{upper-lower+1}

    :Parameters:
      - `x` : [int] :math:`x \in \{lower, lower+1, \ldots, upper-1, upper\}`
      - `lower` : Lower limit.
      - `upper` : Upper limit (upper > lower).

    
File:           ~/anaconda3/envs/study/lib/python3.6/site-packages/pymc/distributions.py
Type:           StochasticMeta


In [5]:
some_variable = pm.DiscreteUniform('discrete_uni_var', 0, 4)
print('stochastic :', some_variable.value)

stochastic : 3


변수를 여러개를 랜덤하게 출력하고 싶을 경우 `size` keyword를 이용하여 독립적인 확률변수의 리스트가 만들어 진다.

In [6]:
betas = pm.Uniform('betas', 0, 1, size=10)
print('betas :', betas.value)
# ??pm.Uniform

betas : [ 0.70306154  0.94839185  0.71569485  0.98129065  0.68459859  0.58179276
  0.16934249  0.5849588   0.72492596  0.76424481]


#### `random()` 호출하기

`stochastic` 변수의 `random()` 메소드를 호출하여 새 난수값을 만들 수 있다.

In [7]:
lambda_1 = pm.Exponential("lambda_1", 1)
lambda_2 = pm.Exponential('lambda_2', 1)
tau = pm.DiscreteUniform('tau', lower=0, upper=10)

print("Initialized values...")
print("lambda_1.value = %.3f" % lambda_1.value)
print("lambda_2.value = %.3f" % lambda_2.value)
print("tau.value = %.3f" % tau.value, "\n")
print(lambda_1.random(), lambda_2.random(), tau.random())
print("After calling random() on the variables...")
print("lambda_1.value = %.3f" % lambda_1.value)
print("lambda_2.value = %.3f" % lambda_2.value)
print("tau.value = %.3f" % tau.value)

Initialized values...
lambda_1.value = 0.439
lambda_2.value = 1.108
tau.value = 5.000 

1.042378189189035 0.4471787834199512 0
After calling random() on the variables...
lambda_1.value = 1.042
lambda_2.value = 0.447
tau.value = 0.000


#### deterministic 변수

deterministic 변수는 아래와 같이 **데코레이터**를 이용해 정의해 줄 수 있다.

```python
@pm.deterministic
def some_deterministic_var(v1=v1, ):
    # type code here
```

In [9]:
type(lambda_1 + lambda_2)

pymc.PyMCObjects.Deterministic

In [ ]:
import numpy as 